## Titanic Data Analysis Assignment

### Part A — Basic\n",

In [ ]:
# Task 1: Load & Inspect

import pandas as pd
import numpy as np
import re

df = pd.read_csv('train.csv')
print("DataFrame Shape:")
print(df.shape)
print("\n")   
print("DataFrame Info:")   
print(df.info())
print("\n") 
print("DataFrame Head (5 rows):")
print(df.head(5))


DataFrame Shape:
(891, 12)


DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


DataFrame Head (5 rows):
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         

In [5]:

# Task 2: Column summary

summary_data = {
        'column_name': df.columns,
        'dtype': df.dtypes,
        '# missing': df.isnull().sum(),
        '# unique values': df.nunique()
    }
summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.sort_values(by='# missing', ascending=False)
print(summary_df)

             column_name    dtype  # missing  # unique values
Cabin              Cabin   object        687              147
Age                  Age  float64        177               88
Embarked        Embarked   object          2                3
PassengerId  PassengerId    int64          0              891
Survived        Survived    int64          0                2
Pclass            Pclass    int64          0                3
Name                Name   object          0              891
Sex                  Sex   object          0                2
SibSp              SibSp    int64          0                7
Parch              Parch    int64          0                7
Ticket            Ticket   object          0              681
Fare                Fare  float64          0              248


In [ ]:
# Task 3: Value counts & proportions

for col in ['Pclass', 'Sex', 'Embarked']:
    print(f"\nValue counts for {col}:")
    print(df[col].value_counts().to_frame())
    print(f"\nPercentages for {col}:")
    print(df[col].value_counts(normalize=True).to_frame())



Value counts for Pclass:
        count
Pclass       
3         491
1         216
2         184

Percentages for Pclass:
        proportion
Pclass            
3         0.551066
1         0.242424
2         0.206510

Value counts for Sex:
        count
Sex          
male      577
female    314

Percentages for Sex:
        proportion
Sex               
male      0.647587
female    0.352413

Value counts for Embarked:
          count
Embarked       
S           644
C           168
Q            77

Percentages for Embarked:
          proportion
Embarked            
S           0.724409
C           0.188976
Q           0.086614


In [9]:
# Task 4: Select & filter

female_firstclass_over_30 = df[(df['Sex'] == 'female') & (df['Pclass'] == 1) & (df['Age'] > 30)]
female_firstclass_over_30_sorted = female_firstclass_over_30.sort_values(by='Fare', ascending=False)

print("Top 10 Female First Class Passengers over 30, sorted by Fare:")
print("\n", female_firstclass_over_30_sorted.head(10))

Top 10 Female First Class Passengers over 30, sorted by Fare:

      PassengerId  Survived  Pclass  \
258          259         1       1   
299          300         1       1   
716          717         1       1   
380          381         1       1   
779          780         1       1   
318          319         1       1   
856          857         1       1   
609          610         1       1   
268          269         1       1   
195          196         1       1   

                                                  Name     Sex   Age  SibSp  \
258                                   Ward, Miss. Anna  female  35.0      0   
299    Baxter, Mrs. James (Helene DeLaudeniere Chaput)  female  50.0      0   
716                      Endres, Miss. Caroline Louise  female  38.0      0   
380                              Bidois, Miss. Rosalie  female  42.0      0   
779  Robert, Mrs. Edward Scott (Elisabeth Walton Mc...  female  43.0      0   
318                           Wick, Miss. M

In [17]:
# Task 5: Basic aggregations

print(f"Mean/Median/Mode of Age (ignoring missing values):")
print(f"Mean: {df['Age'].mean()}")
print(f"Median: {df['Age'].median()}")
print(f"Mode: {df['Age'].mode().values}")
print("\nMean Fare per Pclass:")
print(df.groupby('Pclass')['Fare'].mean().to_frame())
print("\nSurvival Rate:")
print(f"Overall: {df['Survived'].mean()}")
print(f"By Gender:\\n{df.groupby('Sex')['Survived'].mean().to_frame()}")

Mean/Median/Mode of Age (ignoring missing values):
Mean: 29.69911764705882
Median: 28.0
Mode: [24.]

Mean Fare per Pclass:
             Fare
Pclass           
1       84.154687
2       20.662183
3       13.675550

Survival Rate:
Overall: 0.3838383838383838
By Gender:\n        Survived
Sex             
female  0.742038
male    0.188908


### Part B — Intermediate

In [19]:
# Task 1: Missing value imputation

print("Missing Age values before imputation:")
print(df['Age'].isnull().sum())

df['Age_imputed'] = df.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))

print("\nMissing Age values after imputation:")
print(df['Age_imputed'].isnull().sum())

Missing Age values before imputation:
177

Missing Age values after imputation:
0


In [21]:
# Task 2: Feature extraction from text

df['Title'] = df['Name'].apply(lambda x: re.search('([A-Za-z]+)\\.', x).group(1))
df['Title'] = df['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
df['Title'] = df['Title'].replace('Mlle', 'Miss')
df['Title'] = df['Title'].replace('Ms', 'Miss')
df['Title'] = df['Title'].replace('Mme', 'Mrs')
    
print("Counts for each Title:")
print(df['Title'].value_counts().to_frame())

Counts for each Title:
        count
Title        
Mr        517
Miss      185
Mrs       126
Master     40
Rare       23


In [22]:
# Task 3: Family size & new feature (5 pts)

df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
df['IsAlone'] = (df['FamilySize'] == 1)
print("Survival rate by IsAlone:")
print(df.groupby('IsAlone')['Survived'].mean().to_frame())

Survival rate by IsAlone:
         Survived
IsAlone          
False    0.505650
True     0.303538


In [23]:
# Task 4: Pivot table and multi-index groupby

print("Pivot table showing survival rate by Pclass and Sex:")
pivot_table_survival = pd.pivot_table(df, values='Survived', index='Pclass', columns='Sex', aggfunc='mean')
print(pivot_table_survival)
print("\nMulti-index groupby showing average Fare and Age_imputed for Pclass, Embarked:")
groupby_fare_age = df.groupby(['Pclass', 'Embarked']).agg({'Fare': 'mean', 'Age_imputed': 'mean'})
print(groupby_fare_age)

Pivot table showing survival rate by Pclass and Sex:
Sex       female      male
Pclass                    
1       0.968085  0.368852
2       0.921053  0.157407
3       0.500000  0.135447

Multi-index groupby showing average Fare and Age_imputed for Pclass, Embarked:
                       Fare  Age_imputed
Pclass Embarked                         
1      C         104.718529    37.988235
       Q          90.000000    38.500000
       S          70.364862    38.271024
2      C          25.358335    23.617647
       Q          12.350000    38.333333
       S          20.327439    30.355671
3      C          11.214083    21.983636
       Q          11.183393    24.194444
       S          14.644083    25.453258


In [25]:
# Task 5: String cleaning & parsing

df['Cabin'] = df['Cabin'].fillna('Unknown')
df['CabinDeck'] = df['Cabin'].apply(lambda x: x[0])
    
print("Survival rate by CabinDeck:")
print(df.groupby('CabinDeck')['Survived'].mean().to_frame())

Survival rate by CabinDeck:
           Survived
CabinDeck          
A          0.466667
B          0.744681
C          0.593220
D          0.757576
E          0.750000
F          0.615385
G          0.500000
T          0.000000
U          0.299854


### Part C — Advanced

In [29]:
# Task 1: Advanced joins / merges

ticket_counts = df['Ticket'].value_counts().reset_index()
ticket_counts.columns = ['Ticket', 'TicketCount']
df = df.merge(ticket_counts, on='Ticket', how='left')
print("Top 10 tickets by TicketCount:")
print(df.sort_values('TicketCount', ascending=False)[['Ticket', 'TicketCount']].head(10))

Top 10 tickets by TicketCount:
       Ticket  TicketCount
201  CA. 2343            7
13     347082            7
643      1601            7
863  CA. 2343            7
813    347082            7
541    347082            7
169      1601            7
119    347082            7
542    347082            7
610    347082            7


In [30]:
# Task 2: Outlier detection & handling

fare_99th_percentile = df['Fare'].quantile(0.99)
print(f"Fare 99th percentile: {fare_99th_percentile}")
    
df['Fare_outlier'] = df['Fare'] > fare_99th_percentile
    
print("\nMean/Median Fare before handling outliers:")
print(f"Mean: {df['Fare'].mean()}")
print(f"Median: {df['Fare'].median()}")
    
df['Fare_cleaned'] = df['Fare'].apply(lambda x: fare_99th_percentile if x > fare_99th_percentile else x)
    
print("\nMean/Median Fare after replacing outliers with 99th percentile:")
print(f"Mean: {df['Fare_cleaned'].mean()}")
print(f"Median: {df['Fare_cleaned'].median()}")

Fare 99th percentile: 249.00622000000035

Mean/Median Fare before handling outliers:
Mean: 32.204207968574636
Median: 14.4542

Mean/Median Fare after replacing outliers with 99th percentile:
Mean: 31.224767317620657
Median: 14.4542


In [31]:
# Task 3: Complex aggregation with apply

bins = [0, 11, 17, 30, 60, np.inf]
labels = ['Child', 'Teen', 'YoungAdult', 'Adult', 'Senior']
df['AgeGroup'] = pd.cut(df['Age_imputed'], bins=bins, labels=labels, right=True)   
print("Survival rate for each AgeGroup by Sex:")
survival_by_agegroup_sex = df.groupby(['AgeGroup', 'Sex'])['Survived'].mean().to_frame()
print(survival_by_agegroup_sex)

Survival rate for each AgeGroup by Sex:
                   Survived
AgeGroup   Sex             
Child      female  0.593750
           male    0.555556
Teen       female  0.826087
           male    0.136364
YoungAdult female  0.700680
           male    0.135135
Adult      female  0.816514
           male    0.215686
Senior     female  1.000000
           male    0.105263


C:\Users\Kinjal\AppData\Local\Temp\ipykernel_372\1523340558.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  survival_by_agegroup_sex = df.groupby(['AgeGroup', 'Sex'])['Survived'].mean().to_frame()


In [33]:
# Task 4: Multi-step pipeline

df_clean = df.copy()

# Fill missing Embarked with mode
mode_embarked = df_clean['Embarked'].mode()[0]
df_clean['Embarked'] = df_clean['Embarked'].fillna(mode_embarked)

# Impute Age using Pclass+Sex median rule
df_clean['Age'] = df_clean.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))

# Encode Gender to numeric
df_clean['Sex'] = df_clean['Sex'].map({'male': 0, 'female': 1})

# Drop unused columns
df_clean = df_clean.drop(columns=['Name', 'Ticket', 'Cabin', 'Fare_outlier', 'Fare_cleaned', 'Age_imputed', 'Title', 'FamilySize', 'IsAlone', 'CabinDeck', 'AgeGroup', 'TicketCount'])

print("df_clean head:")
print(df_clean.head())

print("\ndf_clean info:")
print(df_clean.info())

df_clean.to_csv('train_cleaned.csv', index=False)

df_clean head:
   PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch     Fare Embarked  \
0            1         0       3    0  22.0      1      0   7.2500        S   
1            2         1       1    1  38.0      1      0  71.2833        C   
2            3         1       3    1  26.0      0      0   7.9250        S   
3            4         1       1    1  35.0      1      0  53.1000        S   
4            5         0       3    0  35.0      0      0   8.0500        S   

   TicketCount_x  TicketCount_y  
0              1              1  
1              1              1  
2              1              1  
3              2              2  
4              1              1  

df_clean info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PassengerId    891 non-null    int64  
 1   Survived       891 non-null    int64  
 2   Pcla

In [34]:
# Task 5: Challenge analysis / storytelling

# Overall survival rate
overall_survival = df['Survived'].mean()
print(f"Overall Survival Rate: {overall_survival:.3f}")
print("\n" + "="*50)

# Individual feature analysis
features_to_analyze = ['Sex', 'Pclass', 'Age_imputed', 'Embarked', 'IsAlone', 'FamilySize']

for feature in features_to_analyze:
    if feature == 'Age_imputed':
        # Create age groups for better analysis
        bins = [0, 12, 18, 35, 60, np.inf]
        labels = ['Child', 'Teen', 'Young Adult', 'Middle Age', 'Senior']
        df['AgeGroup'] = pd.cut(df['Age_imputed'], bins=bins, labels=labels, right=False)
        survival_by_feature = df.groupby('AgeGroup')['Survived'].agg(['count', 'mean']).round(3)
        print(f"\nSurvival by Age Group:")
    elif feature == 'FamilySize':
        # Group family sizes for better analysis
        df['FamilySizeGroup'] = df['FamilySize'].apply(lambda x: 'Alone' if x == 1 else 'Small (2-4)' if x <= 4 else 'Large (5+)')
        survival_by_feature = df.groupby('FamilySizeGroup')['Survived'].agg(['count', 'mean']).round(3)
        print(f"\nSurvival by Family Size:")
    else:
        survival_by_feature = df.groupby(feature)['Survived'].agg(['count', 'mean']).round(3)
        print(f"\nSurvival by {feature}:")
    
    print(survival_by_feature)
    print("-" * 30)
    
print("TWO-FEATURE COMBINATIONS ANALYSIS")
print("=" * 50)

# 1. Sex + Pclass (Classic combination)
print("\n1. SEX + PASSENGER CLASS")
sex_pclass_pivot = pd.pivot_table(df, values='Survived', index='Pclass', columns='Sex', aggfunc=['count', 'mean'])
print("\nCounts:")
print(sex_pclass_pivot['count'])
print("\nSurvival Rates:")
print(sex_pclass_pivot['mean'].round(3))

# Calculate range of survival rates for this combination
sex_pclass_rates = df.groupby(['Sex', 'Pclass'])['Survived'].mean()
print(f"Range of survival rates: {sex_pclass_rates.min():.3f} to {sex_pclass_rates.max():.3f}")
print(f"Difference: {sex_pclass_rates.max() - sex_pclass_rates.min():.3f}")

print("\n" + "-" * 50)

# 2. Sex + Age Group
print("\n2. SEX + AGE GROUP")
sex_age_pivot = pd.pivot_table(df, values='Survived', index='AgeGroup', columns='Sex', aggfunc=['count', 'mean'])
print("\nCounts:")
print(sex_age_pivot['count'])
print("\nSurvival Rates:")
print(sex_age_pivot['mean'].round(3))

sex_age_rates = df.groupby(['Sex', 'AgeGroup'])['Survived'].mean()
print(f"Range of survival rates: {sex_age_rates.min():.3f} to {sex_age_rates.max():.3f}")
print(f"Difference: {sex_age_rates.max() - sex_age_rates.min():.3f}")

print("\n" + "-" * 50)

# 3. Pclass + Age Group
print("\n3. PASSENGER CLASS + AGE GROUP")
pclass_age_pivot = pd.pivot_table(df, values='Survived', index='AgeGroup', columns='Pclass', aggfunc=['count', 'mean'])
print("\nCounts:")
print(pclass_age_pivot['count'])
print("\nSurvival Rates:")
print(pclass_age_pivot['mean'].round(3))

pclass_age_rates = df.groupby(['Pclass', 'AgeGroup'])['Survived'].mean()
print(f"Range of survival rates: {pclass_age_rates.min():.3f} to {pclass_age_rates.max():.3f}")
print(f"Difference: {pclass_age_rates.max() - pclass_age_rates.min():.3f}")

print("\n\nTHREE-FEATURE COMBINATION ANALYSIS")
print("=" * 50)

# The most promising combination: Sex + Pclass + Age Group
print("\nSEX + PASSENGER CLASS + AGE GROUP")
three_way_analysis = df.groupby(['Sex', 'Pclass', 'AgeGroup'])['Survived'].agg(['count', 'mean']).round(3)
print(three_way_analysis)

# Get the range of survival rates
three_way_rates = df.groupby(['Sex', 'Pclass', 'AgeGroup'])['Survived'].mean()
print(f"\nRange of survival rates: {three_way_rates.min():.3f} to {three_way_rates.max():.3f}")
print(f"Difference: {three_way_rates.max() - three_way_rates.min():.3f}")

# Show the extreme cases
print(f"\nHighest survival rate groups:")
top_groups = three_way_rates.nlargest(3)
for idx, rate in top_groups.items():
    print(f"  {idx}: {rate:.3f}")

print(f"\nLowest survival rate groups:")
bottom_groups = three_way_rates.nsmallest(3)
for idx, rate in bottom_groups.items():
    print(f"  {idx}: {rate:.3f}")
    
print("\n\nSTATISTICAL SUMMARY")
print("=" * 50)

# Calculate variance in survival rates for different combinations
combinations = {
    'Sex': df.groupby('Sex')['Survived'].mean(),
    'Pclass': df.groupby('Pclass')['Survived'].mean(),
    'AgeGroup': df.groupby('AgeGroup')['Survived'].mean(),
    'Sex + Pclass': df.groupby(['Sex', 'Pclass'])['Survived'].mean(),
    'Sex + AgeGroup': df.groupby(['Sex', 'AgeGroup'])['Survived'].mean(),
    'Pclass + AgeGroup': df.groupby(['Pclass', 'AgeGroup'])['Survived'].mean(),
    'Sex + Pclass + AgeGroup': df.groupby(['Sex', 'Pclass', 'AgeGroup'])['Survived'].mean()
}

print("Variance in survival rates (higher = better discrimination):")
for combo_name, rates in combinations.items():
    variance = rates.var()
    min_rate = rates.min()
    max_rate = rates.max()
    range_diff = max_rate - min_rate
    print(f"{combo_name:25}: Variance={variance:.4f}, Range={range_diff:.3f} ({min_rate:.3f} to {max_rate:.3f})")

print("\n\nKEY INSIGHTS FROM BEST COMBINATION (Sex + Pclass + AgeGroup)")
print("=" * 65)

# Create a detailed breakdown of the best performers vs worst performers
best_combo_detailed = df.groupby(['Sex', 'Pclass', 'AgeGroup']).agg({
    'Survived': ['count', 'sum', 'mean'],
    'Age_imputed': 'mean',
    'FamilySize': 'mean'
}).round(3)

# Flatten column names
best_combo_detailed.columns = ['Count', 'Survivors', 'Survival_Rate', 'Avg_Age', 'Avg_Family_Size']

# Sort by survival rate
best_combo_detailed = best_combo_detailed.sort_values('Survival_Rate', ascending=False)
print(best_combo_detailed)

# Show groups with sufficient sample size (>= 10 passengers)
print(f"\n\nGroups with >= 10 passengers (more reliable statistics):")
reliable_groups = best_combo_detailed[best_combo_detailed['Count'] >= 10].sort_values('Survival_Rate', ascending=False)
print(reliable_groups)

Overall Survival Rate: 0.384


Survival by Sex:
        count   mean
Sex                 
female    314  0.742
male      577  0.189
------------------------------

Survival by Pclass:
        count   mean
Pclass              
1         216  0.630
2         184  0.473
3         491  0.242
------------------------------

Survival by Age Group:
             count   mean
AgeGroup                 
Child           68  0.574
Teen            45  0.489
Young Adult    513  0.337
Middle Age     239  0.423
Senior          26  0.269
------------------------------

Survival by Embarked:
          count   mean
Embarked              
C           168  0.554
Q            77  0.390
S           644  0.337
------------------------------

Survival by IsAlone:
         count   mean
IsAlone              
False      354  0.506
True       537  0.304
------------------------------

Survival by Family Size:
                 count   mean
FamilySizeGroup              
Alone              537  0.304
Large (5+)       

C:\Users\Kinjal\AppData\Local\Temp\ipykernel_372\3195273242.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  survival_by_feature = df.groupby('AgeGroup')['Survived'].agg(['count', 'mean']).round(3)
C:\Users\Kinjal\AppData\Local\Temp\ipykernel_372\3195273242.py:51: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  sex_age_pivot = pd.pivot_table(df, values='Survived', index='AgeGroup', columns='Sex', aggfunc=['count', 'mean'])
C:\Users\Kinjal\AppData\Local\Temp\ipykernel_372\3195273242.py:51: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=Fa

Variance in survival rates (higher = better discrimination):
Sex                      : Variance=0.1530, Range=0.553 (0.189 to 0.742)
Pclass                   : Variance=0.0379, Range=0.387 (0.242 to 0.630)
AgeGroup                 : Variance=0.0145, Range=0.304 (0.269 to 0.574)
Sex + Pclass             : Variance=0.1327, Range=0.833 (0.135 to 0.968)
Sex + AgeGroup           : Variance=0.1096, Range=0.864 (0.136 to 1.000)
Pclass + AgeGroup        : Variance=0.0810, Range=0.906 (0.094 to 1.000)
Sex + Pclass + AgeGroup  : Variance=0.1679, Range=1.000 (0.000 to 1.000)


KEY INSIGHTS FROM BEST COMBINATION (Sex + Pclass + AgeGroup)
                           Count  Survivors  Survival_Rate  Avg_Age  \
Sex    Pclass AgeGroup                                                
male   1      Child            3          3          1.000    5.307   
female 2      Teen             4          4          1.000   15.250   
male   2      Child            9          9          1.000    2.259   
       1  

Summary and Conclusion
Based on this comprehensive analysis, Sex + Passenger Class + Age Group emerges as the most predictive combination of features for Titanic survival. This three-feature combination shows the highest variance in survival rates (ranging from 0% to 100% in some subgroups), indicating strong discriminative power. The analysis reveals clear patterns: first-class female passengers, especially those in younger age groups, had dramatically higher survival rates (often above 90%), while third-class male passengers across most age groups had survival rates below 20%. The "women and children first" maritime protocol is clearly evident, but passenger class significantly modified these effects, with socioeconomic status playing a crucial role in determining who had access to lifeboats. This combination captures both the social dynamics (gender and class hierarchies) and biological factors (age-based vulnerability) that determined survival during this maritime disaster.